In [75]:
import numpy as np

In [76]:
top_filename = "../data/apo_solv.top"
with open(top_filename) as file:
    lines = file.readlines()
    ligand_indices = [line.split()[0] for line in lines if "PT1" in line and line[0] != ";"]
    ligand_comment_index = [[index for index in range(len(lines)) if "PT1" in lines[index]][0]]


In [123]:
print(ligand_indices)

['3418', '3419', '3420', '3421', '3422', '3423', '3424', '3425', '3426', '3427', '3428', '3429', '3430', '3431', '3432', '3433', '3434', '3435', '3436', '3437', '3438', '3439', '3440', '3441']


In [77]:
with open(top_filename, "r") as file:
    lines = file.readlines()
    find_index = lambda top_type: [index for index in range(len(lines)) if top_type in lines[index]]
    bond_indices = find_index("bonds")
    pair_indices = find_index("pairs")
    angle_indices = find_index("angles")
    dihedral_indices = find_index("dihedrals")
    moleculetype_indices = find_index("moleculetype")


In [78]:
print(f"ligand residues start from: {ligand_comment_index}")
print(f"bond indices: {bond_indices}")
print(f"pair indices: {pair_indices}")
print(f"angle indices: {angle_indices}")
print(f"dihedral indices: {dihedral_indices}")
print(f"moleculetype_indices: {moleculetype_indices}")

ligand residues start from: [3736]
bond indices: [3762, 38394]
pair indices: [17, 7254]
angle indices: [16364]
dihedral indices: [22676]
moleculetype_indices: [81, 38371, 38381]


In [83]:
unsorted_elements = ligand_comment_index + bond_indices + pair_indices + angle_indices + dihedral_indices + moleculetype_indices
all_elements = sorted(unsorted_elements)
keys = ["pairs1", "types1", "ligand", "bonds1", "pairs2", "angles", "dihedrals", "types2", "types3", "bonds2"]
top_dict = {}
for i in range(len(keys)):
    top_dict[keys[i]] = all_elements[i]
print(top_dict)

{'pairs1': 17, 'types1': 81, 'ligand': 3736, 'bonds1': 3762, 'pairs2': 7254, 'angles': 16364, 'dihedrals': 22676, 'types2': 38371, 'types3': 38381, 'bonds2': 38394}


In [116]:
bond_lines = [lines[line_nr] for line_nr in range(len(lines)) if line_nr >= top_dict["bonds1"] and line_nr < top_dict["pairs2"]]
pair_lines = [lines[line_nr] for line_nr in range(len(lines)) if line_nr >= top_dict["pairs2"] and line_nr < top_dict["angles"]]
angle_lines = [lines[line_nr] for line_nr in range(len(lines)) if line_nr >= top_dict["angles"] and line_nr < top_dict["dihedrals"]]
dihedral_lines = [lines[line_nr] for line_nr in range(len(lines)) if line_nr >= top_dict["dihedrals"] and line_nr < top_dict["types2"]]


In [122]:
outfile = "../data/test.top"
with open(outfile, "w") as file:
    for line_nr in range(len(lines)):
        if line_nr < top_dict["ligand"]:
            file.write(lines[line_nr])
    for bond_line in bond_lines:
        check_line = bond_line.split()
        if len(check_line) > 0:
            ai, aj = check_line[0], check_line[1]
            if ai not in ligand_indices and aj not in ligand_indices:
                file.write(bond_line)
    for pair_line in pair_lines:
        check_line = pair_line.split()
        if len(check_line) > 0:
            ai, aj = check_line[0], check_line[1]
            if ai not in ligand_indices and aj not in ligand_indices:
                file.write(pair_line)
    for angle_line in angle_lines:
        check_line = angle_line.split()
        if len(check_line) > 0:
            ai, aj, ak = check_line[0], check_line[1], check_line[2]
            if ai not in ligand_indices and aj not in ligand_indices and ak not in ligand_indices:
                file.write(angle_line)
    for dihedral_line in dihedral_lines:
        check_line = dihedral_line.split()
        if len(check_line) > 0 and len(check_line) < 4:
            file.write(dihedral_line)
        elif len(check_line) >= 4:
            ai, aj, ak, al = check_line[0], check_line[1], check_line[2], check_line[3]
            if ai not in ligand_indices and aj not in ligand_indices and ak not in ligand_indices and al not in ligand_indices:
                file.write(dihedral_line)
    for line_nr in range(len(lines)):
        if line_nr >= top_dict["types2"]:
            file.write(lines[line_nr]) 
        